### Importing the libraries

In [48]:
%pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [49]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [50]:
tf.__version__

'2.11.0'

## Data Preprocessing

### Importing the dataset

In [51]:
ds = pd.read_csv('Churn_Modelling.csv')
X = ds.iloc[:, 3:-1].values
y = ds.iloc[:, -1].values
print(X)
print(y)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]
[1 0 1 ... 1 1 0]


### Encoding categorical data

Label Encoding the "Gender" column

In [52]:
from sklearn.preprocessing import LabelEncoder
labelEncoder = LabelEncoder()
X[:, 2] = labelEncoder.fit_transform(X[:, 2])
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One Hot Encoding the "Geography" column

In [53]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


### Splitting the dataset into the Training and Test set

In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

### Feature Scaling

In [55]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

## Building the ANN

### Initializing the ANN

In [56]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [57]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [58]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer

In [59]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Training the ANN

### Compiling the ANN

In [60]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the ANN on the Training set

In [61]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5145 - accuracy: 0.7934
Epoch 2/100
250/250 [==============================] - 0s 907us/step - loss: 0.4538 - accuracy: 0.8035
Epoch 3/100
250/250 [==============================] - 0s 883us/step - loss: 0.4377 - accuracy: 0.8111
Epoch 4/100
250/250 [==============================] - 0s 913us/step - loss: 0.4300 - accuracy: 0.8135
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4237 - accuracy: 0.8190
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4180 - accuracy: 0.8225
Epoch 7/100
250/250 [==============================] - 0s 952us/step - loss: 0.4124 - accuracy: 0.8238
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4081 - accuracy: 0.8249
Epoch 9/100
250/250 [==============================] - 0s 884us/step - loss: 0.4034 - accuracy: 0.8273
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.398

## Making the predictions and evaluating the model

### Predicting the result of a single observation

**PROBLEM**

Use the ANN model to predict if the customer with the following informations will leave the bank or not: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card ? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

**Solution**

In [62]:
print(ann.predict(scaler.transform([[1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]])) > 0.5)

1/1 [==============================] - 0s 96ms/step
[[False]]


Therefore, our ANN model predicts that this customer stays in the bank!

**Important note 1:** Notice that the values of the features were all input in a double pair of square brackets. That's because the "predict" method always expects a 2D array as the format of its inputs. And putting our values into a double pair of square brackets makes the input exactly a 2D array.

**Important note 2:** Notice also that the "France" country was not input as a string in the last column but as "1, 0, 0" in the first three columns. That's because of course the predict method expects the one-hot-encoded values of the state, and as we see in the first row of the matrix of features X, "France" was encoded as "1, 0, 0". And be careful to include these values in the first three columns, because the dummy variables are always created in the first columns.

### Predicting the Test set results

In [63]:
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

63/63 [==============================] - 0s 746us/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Confusion Matrix

In [64]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1516   79]
 [ 203  202]]


0.859